# ランキングの取得

- 繰り返しのデータがある場合はまず1つのデータから情報を抽出 → それを元にループ処理にする、という段階を踏むべし。また事前に同じフォーマットであることを確認しておく。
- タグは複数回使われる可能性が高いので親の構造から辿っていくと良い。`element`, `elements` の使い分けに注意。

In [1]:
from selenium import webdriver
browser = webdriver.Chrome()
browser.get('https://scraping-for-beginner.herokuapp.com/ranking/')

## 個別取得

In [2]:
elem_rankingBox = browser.find_element_by_class_name('u_areaListRankingBox')
elem_rankingBox.text

'1\n観光地 1\n4.7\n楽しさ\n4.6\n満喫できた\n人混みの多さ\n4.5\n時間帯によって混雑具合は違った\n景色\n4.9\n景色に魅了された\nアクセス\n4.2\n船で2時間ほどであった'

In [3]:
elem_title = elem_rankingBox.find_element_by_class_name('u_title')
elem_title.text

'1\n観光地 1'

In [4]:
title = elem_title.text.split('\n')[1]
title

'観光地 1'

In [5]:
rankBox = browser.find_element_by_class_name('u_rankBox').find_element_by_class_name('evaluateNumber')
rankBox.text

'4.7'

In [6]:
elem_tipsItem = browser.find_element_by_class_name('u_categoryTipsItem')
elem_tipsItem.text

'楽しさ\n4.6\n満喫できた\n人混みの多さ\n4.5\n時間帯によって混雑具合は違った\n景色\n4.9\n景色に魅了された\nアクセス\n4.2\n船で2時間ほどであった'

In [7]:
print(elem_tipsItem.find_elements_by_class_name('is_rank')[0].find_element_by_class_name('evaluateNumber').text)
print(elem_tipsItem.find_elements_by_class_name('is_rank')[1].find_element_by_class_name('evaluateNumber').text)
print(elem_tipsItem.find_elements_by_class_name('is_rank')[2].find_element_by_class_name('evaluateNumber').text)
print(elem_tipsItem.find_elements_by_class_name('is_rank')[3].find_element_by_class_name('evaluateNumber').text)

4.6
4.5
4.9
4.2


## ページ内一括取得

In [8]:
elems_rankingBox = browser.find_elements_by_class_name('u_areaListRankingBox')
len(elems_rankingBox)

10

In [9]:
titles = []
for elem_rankingBox in elems_rankingBox:
    elem_title = elem_rankingBox.find_element_by_class_name('u_title')
    title = elem_title.text.split('\n')[1]
    titles.append(title)

titles

['観光地 1',
 '観光地 2',
 '観光地 3',
 '観光地 4',
 '観光地 5',
 '観光地 6',
 '観光地 7',
 '観光地 8',
 '観光地 9',
 '観光地 10']

In [10]:
elems_rankBox = browser.find_elements_by_class_name('u_rankBox')
ranks = [float(elem_rankBox.find_element_by_class_name('evaluateNumber').text) for elem_rankBox in elems_rankBox]
ranks

[4.7, 4.7, 4.6, 4.5, 4.5, 4.4, 4.3, 4.3, 4.2, 4.1]

In [11]:
elems_tipsItem = browser.find_elements_by_class_name('u_categoryTipsItem')

evaluateNumbers = []
for elem_tipsItem in elems_tipsItem:
    elems_rank = elem_tipsItem.find_elements_by_class_name('is_rank')
    _evaluateNumbers = []
    for elem_rank in elems_rank:
        _evaluateNumbers.append(float(elem_rank.find_element_by_class_name('evaluateNumber').text))
    evaluateNumbers.append(_evaluateNumbers)
evaluateNumbers

[[4.6, 4.5, 4.9, 4.2],
 [4.6, 4.5, 4.9, 4.2],
 [4.5, 4.4, 4.8, 4.1],
 [4.4, 4.4, 4.8, 4.0],
 [4.4, 4.3, 4.7, 4.0],
 [4.3, 4.3, 4.7, 3.9],
 [4.2, 4.2, 4.6, 3.8],
 [4.2, 4.2, 4.6, 3.8],
 [4.1, 4.1, 4.5, 3.7],
 [4.0, 4.1, 4.4, 3.6]]

## ページング

In [12]:
def get_titles():
    _titles = []
    elems_rankingBox = browser.find_elements_by_class_name('u_areaListRankingBox')
    for elem_rankingBox in elems_rankingBox:
        elem_title = elem_rankingBox.find_element_by_class_name('u_title')
        title = elem_title.text.split('\n')[1]
        _titles.append(title)
    return _titles

In [13]:
def get_ranks():
    elems_rankBox = browser.find_elements_by_class_name('u_rankBox')
    return [float(elem_rankBox.find_element_by_class_name('evaluateNumber').text) for elem_rankBox in elems_rankBox]

In [14]:
def get_categories():
    elems_tipsItem = browser.find_elements_by_class_name('u_categoryTipsItem')

    evaluateNumbers = []
    for elem_tipsItem in elems_tipsItem:
        elems_rank = elem_tipsItem.find_elements_by_class_name('is_rank')
        _evaluateNumbers = []
        for elem_rank in elems_rank:
            _evaluateNumbers.append(float(elem_rank.find_element_by_class_name('evaluateNumber').text))
        evaluateNumbers.append(_evaluateNumbers)
    return evaluateNumbers

In [15]:
titles = []
ranks = []
categories = []

for page in range(1, 4):
    url = 'https://scraping-for-beginner.herokuapp.com/ranking/?page={}'.format(page)
    browser.get(url)

    titles.append(get_titles())
    ranks.append(get_ranks())
    categories.append(get_categories())

In [16]:
def flatten(nested_list):
    return sum(nested_list, [])

titles = flatten(titles)
ranks = flatten(ranks)
categories = flatten(categories)

In [17]:
titles

['観光地 1',
 '観光地 2',
 '観光地 3',
 '観光地 4',
 '観光地 5',
 '観光地 6',
 '観光地 7',
 '観光地 8',
 '観光地 9',
 '観光地 10',
 '観光地 11',
 '観光地 12',
 '観光地 13',
 '観光地 14',
 '観光地 15',
 '観光地 16',
 '観光地 17',
 '観光地 18',
 '観光地 19',
 '観光地 20',
 '観光地 21',
 '観光地 22',
 '観光地 23',
 '観光地 24',
 '観光地 25',
 '観光地 26',
 '観光地 27',
 '観光地 28',
 '観光地 29',
 '観光地 30']

In [18]:
ranks

[4.7,
 4.7,
 4.6,
 4.5,
 4.5,
 4.4,
 4.3,
 4.3,
 4.2,
 4.1,
 4.1,
 4.0,
 3.9,
 3.9,
 3.8,
 3.7,
 3.7,
 3.6,
 3.5,
 3.5,
 3.4,
 3.3,
 3.3,
 3.2,
 3.1,
 3.1,
 3.0,
 2.9,
 2.9,
 2.8]

In [19]:
categories

[[4.6, 4.5, 4.9, 4.2],
 [4.6, 4.5, 4.9, 4.2],
 [4.5, 4.4, 4.8, 4.1],
 [4.4, 4.4, 4.8, 4.0],
 [4.4, 4.3, 4.7, 4.0],
 [4.3, 4.3, 4.7, 3.9],
 [4.2, 4.2, 4.6, 3.8],
 [4.2, 4.2, 4.6, 3.8],
 [4.1, 4.1, 4.5, 3.7],
 [4.0, 4.1, 4.4, 3.6],
 [4.0, 4.0, 4.4, 3.6],
 [3.9, 4.0, 4.3, 3.5],
 [3.8, 3.9, 4.3, 3.4],
 [3.8, 3.9, 4.2, 3.4],
 [3.7, 3.8, 4.2, 3.3],
 [3.6, 3.8, 4.1, 3.2],
 [3.6, 3.7, 4.1, 3.2],
 [3.5, 3.7, 4.0, 3.1],
 [3.4, 3.6, 3.9, 3.0],
 [3.4, 3.6, 3.9, 3.0],
 [3.3, 3.5, 3.8, 2.9],
 [3.2, 3.5, 3.8, 2.8],
 [3.2, 3.4, 3.7, 2.8],
 [3.1, 3.4, 3.7, 2.7],
 [3.0, 3.3, 3.6, 2.6],
 [3.0, 3.3, 3.6, 2.6],
 [2.9, 3.2, 3.5, 2.5],
 [2.8, 3.2, 3.4, 2.4],
 [2.8, 3.1, 3.4, 2.4],
 [2.7, 3.1, 3.3, 2.3]]

## DataFrame の利用・CSV出力

In [20]:
import pandas as pd

In [21]:
df = pd.DataFrame()

In [22]:
df_categories = pd.DataFrame(categories, columns=['楽しさ', '人混みの多さ', '景色', 'アクセス'])

In [23]:
df['観光地名'] = titles
df['総合評価'] = ranks
df = pd.concat([df, df_categories], axis=1)
df

,観光地名,総合評価,楽しさ,人混みの多さ,景色,アクセス
0,観光地 1,4.7,4.6,4.5,4.9,4.2
1,観光地 2,4.7,4.6,4.5,4.9,4.2
2,観光地 3,4.6,4.5,4.4,4.8,4.1
3,観光地 4,4.5,4.4,4.4,4.8,4.0
4,観光地 5,4.5,4.4,4.3,4.7,4.0
5,観光地 6,4.4,4.3,4.3,4.7,3.9
6,観光地 7,4.3,4.2,4.2,4.6,3.8
7,観光地 8,4.3,4.2,4.2,4.6,3.8
8,観光地 9,4.2,4.1,4.1,4.5,3.7
9,観光地 10,4.1,4.0,4.1,4.4,3.6


In [24]:
df.to_csv('観光地情報.csv', index=False)